In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch

In [2]:
%config Completer.use_jedi = False

In [3]:
torch.manual_seed(1)

In [4]:
train = pd.read_csv("./2021-ai-w6-p2/train.csv")
test = pd.read_csv("./2021-ai-w6-p2/X_test.csv")
submit = pd.read_csv("./2021-ai-w6-p2/sample_submit.csv")

In [5]:
print(train.head())
print(test.head())
print(submit.head())

   Day  Hour  Minute  DHI  DNI   WS     RH   T  TARGET
0    0     0       0    0    0  1.5  69.08 -12     0.0
1    0     0      30    0    0  1.5  69.06 -12     0.0
2    0     1       0    0    0  1.6  71.78 -12     0.0
3    0     1      30    0    0  1.6  71.75 -12     0.0
4    0     2       0    0    0  1.6  75.20 -12     0.0
   ID   Day  Hour  Minute  DHI  DNI   WS     RH   T
0   0  1043     7      30   34  584  2.0  63.02   3
1   1  1043     8       0   45  730  2.3  61.54   5
2   2  1043     8      30   52  817  2.4  57.34   6
3   3  1043     9       0   57  878  2.6  52.46   8
4   4  1043     9      30   61  917  2.2  45.74  10
   ID  TARGET
0   0       0
1   1       0
2   2       0
3   3       0
4   4       0


In [6]:
# preprocessing
x_train = np.array(train.drop(['TARGET'],axis =1 ))
y_train = np.array(train['TARGET'])
x_test = np.array(test.drop(['ID'],axis=1))

In [7]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
scaler = StandardScaler()
le = LabelEncoder()

In [8]:
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()
y_train = torch.Tensor(y_train).cuda()

In [10]:
y_train = y_train.unsqueeze(1)
print(x_train.shape, y_train.shape)

torch.Size([9999, 8]) torch.Size([9999, 1])


In [11]:
layer1 = torch.nn.Linear(8,8,bias= True).cuda()
layer2 = torch.nn.Linear(8,8,bias = True).cuda()
layer3 = torch.nn.Linear(8,4,bias = True).cuda()
layer4 = torch.nn.Linear(4,8,bias = True).cuda()
layer5 = torch.nn.Linear(8,1,bias = True).cuda()

model = torch.nn.Sequential(layer1,layer2,layer3,layer4,layer5).cuda()

In [12]:
optim = torch.optim.SGD(model.parameters(),lr = 1e-4)
loss = torch.nn.MSELoss()
epochs = 1000

In [13]:
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = model(x_train)
    cost = loss(h,y_train)
    cost.backward()
    optim.step()
    if epoch % (epochs/10) == 0:
        print("epochs =", str(epochs), "| epoch =", str(epoch), "| cost =", cost)

epochs = 1000 | epoch = 0 | cost = tensor(996.4373, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 100 | cost = tensor(937.8584, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 200 | cost = tensor(140.3736, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 300 | cost = tensor(66.5223, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 400 | cost = tensor(60.2817, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 500 | cost = tensor(59.4148, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 600 | cost = tensor(59.2310, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 700 | cost = tensor(59.1640, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 800 | cost = tensor(59.1348, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 900 | cost = tensor(59.1216, device='cuda:0', grad_fn=<MseLossBackward>)
epochs = 1000 | epoch = 1000 

In [14]:
with torch.no_grad():
    predict = model(x_test)
    
predict.cpu().detach().numpy()

array([[41.159855 ],
       [49.86627  ],
       [55.473434 ],
       ...,
       [ 7.074028 ],
       [ 6.9251003],
       [ 7.0182023]], dtype=float32)